In [1]:
import pandas as pd     #  数据分析包，一维 series，二维 DataFrame(表格), 三维 panel
import numpy as np       #数组的形式  array
import os
import sys
import time
import json

from math import sin, asin, cos, radians, fabs, sqrt

In [2]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a+')

    def write(self, message):
        self.terminal.write(message)
        self.terminal.flush()
        self.log.write(message)
        self.log.flush()

    def flush(self):
        pass

#logfile_3 =[".\\log\\stay_point_exact_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
#sys.stdout = Logger(logfile_3[0], sys.stdout)
#filename2 =[".\\log\\a_stderr_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log_file']
    
#sys.stdout = Logger(".\\a_stdout.log", sys.stdout)
#sys.stderr = Logger(".\\a_stderr.log_file", sys.stderr)             # redirect std err, if necessary

#sys.stderr = Logger(filename2[0], sys.stderr)   

In [3]:
def write_sp_txt(st_p,filename):
    file = open(filename, 'a+')
    #file.write(str(st_p).decode('unicode_escape'))
    file.write(str(st_p))
    file.close()
    
    
#def write_staypoint(st_p,filename):
    
    #filename = ".\\results.csv"
        #filename =[".\\POI\\a_stdout_" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
#    file = open(filename, 'a+')
        #doc.write(codecs.BOM_UTF8)
        #for ax in json_sel:
        #    doc.write(ax)
        #    doc.write('\n')
#    for i in range(0,len(st_p)):
#        file.write(st_p[i])
#        file.write("\n")
#    file.close() 

In [4]:
# beijing_6huan(tra) 取北京六环内的点
# input 轨迹tra     DataFrame
# output  轨迹tra， 去掉六环外的点    DataFrame
def beijing_6huan(tra):
    
    N_min = 39.695599
    N_max = 40.183304
    E_min = 116.103444
    E_max = 116.719754
    i = 0
    while i < len(tra):
    #for i in range(0, len(tra)):         #range(low, high) 只适用于整数，[low, high)
        a = tra.iloc[i][0]     #二维数据 DataFrame 数据的索引方式，按下标索引
        b = tra.iloc[i][1]
        if a <= N_min or a >= N_max or b <= E_min or b >= E_max:
            #print("出现超过北京六环的位置点，索引为： ", i+7 ,"\n")
            tra.drop(tra.index[i], inplace = True)        #按行号删除  i是实际的行号，tra.index 是原文件的行号
        else:
            i += 1
            
    return tra

In [2]:
#   Dis()  根据经纬度计算距离 distance(a)
#     input:  a  2*2 两个点的经纬度坐标；
#     output:  距离  m

################    测试用例   #################
#a = [[40.009532,116.322153],[40.009493,116.322103]]
#b = pd.DataFrame(a)
#Dis(b)

EARTH_RADIUS = 6371.004           #地球平均半径，6371km
 
def hav(theta):
    s = sin(theta / 2)
    return s * s
 
def Dis(a):        #测量两个地理位置之前的距离
#def get_distance_hav(lat0, lng0, lat1, lng1):
    """用haversine公式计算球面两点间的距离。"""
    # 经纬度转换成弧度
    lat0 = radians(a.iloc[0,0])
    lat1 = radians(a.iloc[1,0])
    lng0 = radians(a.iloc[0,1])
    lng1 = radians(a.iloc[1,1])
 
    dlng = fabs(lng0 - lng1)
    dlat = fabs(lat0 - lat1)
    h = hav(dlat) + cos(lat0) * cos(lat1) * hav(dlng)
    dist = 2 * EARTH_RADIUS * asin(sqrt(h))*1000
 
    return dist

In [3]:
a = [[40.009532,116.322153],[40.009493,116.322103]]
b = pd.DataFrame(a)     #从数组转化成 DataFrame 形式
Dis(b)

6.077853260340526

In [6]:
#  两个位置的时间差函数 Int(b)
#    input  b, [2*1] DataFrame;   
#    Output,   t  时间差， 单位为min 分钟

def Int(b):
    t = (b.iloc[1]-b.iloc[0])*24*60     #时间差，单位是  min分钟
    
    return t

In [7]:
# single_stay_point(tra, D, T)      单文件停留点检测
#   Input 一条轨迹, 距离阈值，时间阈值       DataFrame
#   Ouput  tra, stay point索引list. 每个停留点由 [i,j-1] 起止的位置索引 ,j-1是最后一个在距离阈值范围内的点
def single_stay_point(tra, D, T):
    
#D = 10         #  距离阈值  定义为10米
#T = 10         # 时间阈值  定义为10分钟
#stay_point = np.array([[ 0,0 ]])       # 创建stay_point   array, # 第一行 [0, 0] 作为标题行。
    num1 = len(tra)
    st_p_i = []    # stay_point 索引list; 以 空list 形式定义 返回stay point 的索引;  无标题行,  
    num = 0    # stay_point 的数量
    num2 = 0    # 异常点 的数量
    i = 0
    j = i+2

    while j < len(tra)-1:                 #tra 长度是变化的
        
        #print("查看当前起点位置",tra.iloc[i].name+7,"\n",tra.iloc[i])
        A = tra.iloc[[i,j-1],[0,1]]      #两个位置距离    DataFrame    在j-1位置赋初值， A 是2*2 的经纬度
        B = tra.iloc[[i,j-1],4]      #两个位置的时间 DataFrame   #在j-1位置赋初值，B 是2*1 的两个时间
        a = Dis(A)
        b = Int(B)
    
        A1 = tra.iloc[[i,j],[0,1]]
        A2 = tra.iloc[[i,j+1],[0,1]]
        a1 = Dis(A1)
        a2 = Dis(A2)
    
        if a < D and b>= T:

            if a1 >= D and a2 >= D:
            #c = np.array([[i,j-1]]）            # 把每个stay point的起止位置索引作为一行，并入;
            #stay_point = np.r_[stay_point,c]        #  实现 np.r_[] ：把两个array合并
                #print("stay 前起点：", tra.iloc[i].name+7)
                num += 1        
                c = [i, j-1]   
                st_p_i.append(c)
                print("出现 stay point！！！ 现有", num ,"个stay point；" ,"  停留位置索引范围为： ",  \
                      tra.iloc[i].name+7,"--", tra.iloc[j-1].name+7)     
                # "最后一个停留位置信息为： \n",tra.iloc[j-1])       .name s是什么意思啊？？？？？
                # 停留范围  i--j-1, +7 是因为文件头6行，索引从1开始
                i = j
                j += 2
                #print("stay 后起点：", tra.iloc[i].name+7)
            elif a1 >= D and a2 < D:
                num2 += 1                 #异常点 数量
                #print("异常前起点：", tra.iloc[i].name+7)
                #print("出现 异常点，在 j 处删除，点索引为： ",tra.iloc[j].name+7, "\n",tra.iloc[j])
                #tra = tra.drop(j)      # 此处tra (左边的) 是作为函数内局部变量使用， 并没有改变全局变量 tra (函数外的)
                tra.drop(tra.index[j], inplace = True)
                #print("异常后起点：", tra.iloc[i].name+7)
                
            else:
                j += 1
                     
        elif a >= D and b < T:
            if a1 < D:
                #print("异常前的起点：", tra.iloc[i].name+7)
                #print("出现 异常点，在j-1处，点索引为： ", tra.iloc[j-1].name+7,  "\n",tra.iloc[j-1])
                num2 += 1                 #异常点 数量
 
                #tra = tra.drop(j-1)
                #tra.drop(n-1, inplace = True)        这个按索引删除的   记住这个大错误，，，妈的
                tra.drop(tra.index[j-1], inplace = True)
                #print("异常后的起点：", tra.iloc[i].name+7)
            
            else:
                #print("Moving points, 下阶段索引起始位置为： ", tra.iloc[j-1].name+7)
                i = j-1
                j += 1
                 
        elif a >= D and b>=T:
            #print("间断前起点：", tra.iloc[i].name+7)
            #print("出现 间断点，在j-1处跳过，点索引为： ", tra.iloc[j-1].name+7, "\n",tra.iloc[j-1])
            i = j-1
            j += 1
            #print("间断后起点：", tra.iloc[i].name+7)
        else:
            j += 1

    #print("i: ", i,"j: ",j, len(tra))
    #print("本次执行到索引： ", tra.iloc[i].name+7,"；  len(tra) = ", num1+6)
    #print("本次执行共有stay point:",num,"个；  有异常点：",num2,"个")
    return tra, st_p_i

In [8]:
################################   转换轨迹段 到 停留点
# 计算停留点 stay_point 的平均地理位置 及 停留时间
# input:  一个 stay_point 轨迹段，       所有位置 及 访问时刻  [[Lat, Long, Time],[],[]]
# output:  这个stay_point 的平均地理位置 及 停留时长 [Lat, Long, Duration]   单位， 分钟/min
def mean_point(a):
    b = 0
    c = 0
    for i in range(0, len(a)):
        b = b + a[i][0]
        c = c + a[i][1]
    
    b = b / len(a)
    c = c / len(a)
    
    d = (a[len(a)-1][2] -a[0][2]) * 24*60 
    
    return [b, c, d]

In [9]:
# 计算每一个 第二类 stay_point 的地理位置范围 及 停留时间
# input:  一个 stay_point 的所有位置 及 访问时刻  [[Lat, Long, Time],[],[]]
# output:  这个stay_point 的地理范围 及 停留时长 [[max（Lat），min(Lat), max(Long), min(Long), Time],[],[]]   单位， 分钟/min
def mean_point_2(a):
    b = [x[0] for x in a]      #  list 中取列的方法，  也可以试试转换类型
    c = [y[1] for y in a]
    b1 = min(b)
    b2 = max(b)
    c1 = min(c)
    c2 = max(c)
    
    d = (a[len(a)-1][2] - a[0][2]) * 24*60 
    
    return [b1, b2, c1, c2, d]

In [10]:
# 取文件里所有的 第一类stay_point;  最终以 st_p 返回所有 stay_point 的平均位置 及停留时间
# input: tra 轨迹, D 距离阈值 单位 米/m,  T 时间阈值 单位 分钟/min
# output, st_p list    文件内所有第一类stay_point的平均地理位置 及 停留时间.[[Lat, Long, Time],[],[]]

def single_1st_stay(tra1, D1, T1):     
    #st_p_i = []     # 返回文件内 每个stay_point 的起止索引
    [tra, st_p_i] = single_stay_point(tra1, D1, T1)       # 一个文件里所有的 第一类stay_point;  返回所有 stay_point 起止位置索引列表, [[i,j],,,,[i,j]]
    st_p = []       #  所有第一类 stay_point 的地理位置（经纬度）  及停留时间,  以列表形式保存
    for k in range(0, len(st_p_i)):       # len(st_p_i)表示stay_point的个数;   k 表示 stay_point 的索引
        st_p_l = []                     # 为每一个 stay_point，准备一个容器， 装载stay point内位置序列
        for h in range(st_p_i[k][0],st_p_i[k][1]+1):      # range(low, high), 代表的是第k个 stay_point 的起止位置的索引
            d = [tra.iloc[h][0], tra.iloc[h][1], tra.iloc[h][4]]
            st_p_l.append(d)           # 取出第k个 stay_point 所有位置点，组成一个list
        f = mean_point(st_p_l)         # 计算第k个 stay_point 平均位置点（经纬度）   ****及停留时间
        st_p.append(f)                 # 以 st_p list 返回 所有 stay point
    
    return st_p    #以3元组列表形式 返回所有stay point, [[Lat, Long, Duration],[],[]]

In [11]:
# 一个文件里所有的 第二类stay_point;  最终以 st_p 返回所有 stay_point 的区域 及停留时间
# input: tra 轨迹, D 距离阈值 单位 米/m,  T 时间阈值 单位 分钟/min
# output, st_p list  文件内所有第二类stay_point的地理区域（5元组） 及 停留时间.[[max（Lat），min(Lat), max(Long), min(Long), Time],[],[]]

def single_2st_stay(tra1, D2, T2):
    #st_p_i = []           # 返回每一个 stay_point 的起止索引
    [tra, st_p_i] = single_stay_point(tra1, D2, T2)       # 一个文件里所有的 第一类stay_point;  返回所有 stay_point 起止位置索引列表, [[i,j],,,,[i,j]]
    st_p = []       #  所有第一类 stay_point 的地理位置（经纬度）  及停留时间,  以列表形式保存
    for k in range(0, len(st_p_i)):       # len(st_p_i)表示stay_point的个数;   k 表示 stay_point 的索引
        st_p_l = []                     # 每一个 stay_point，准备一个list容器
        for h in range(st_p_i[k][0],st_p_i[k][1]+1):      # range(low, high), 代表的是第k个 stay_point 的起止位置的索引
            d = [tra.iloc[h][0], tra.iloc[h][1], tra.iloc[h][4]]
            st_p_l.append(d)           # 取出第k个 stay_point 所有位置点，组成一个list
        f = mean_point_2(st_p_l)         # 计算第k个 stay_point 平均位置范围（经纬度） 及停留时间， 【5元组】
        st_p.append(f)                 
    
    return st_p       # 以5元组列表 st_p  返回 所有 stay point

In [3]:
b = open("./st_p_2-20190726-11545.txt", "r")
out = b.read()
test =  json.loads(out)
b.close()

In [4]:
test[0][0][0][0] 

[40.008668, 116.321446, 39744.4063078704]

In [12]:
####################    总体实验___输出停留点所有轨迹点    #########################################
# user = "000"      # 字符串    user user_m    user_f  都是字符串
st_p_1 = []     #第一类停留点list
st_p_2 = []     #第二类停留点list
a = 0
b = 0
path_info = "H:\Geolife Trajectories 1.3\Data\\"
#user_m = str(int(user)+1) 
#user_f = user_m.zfill(3)
user_list = os.listdir(path_info)

filename1 = ".\\Stay_point\\" + "st_p_1-" +str(time.strftime("%Y%m%d-%H%M%S")) + '.txt'
filename2 = ".\\Stay_point\\" + "st_p_2-" +str(time.strftime("%Y%m%d-%H%M%S")) + '.txt'
logfile_3 =[".\\Stay_point\\" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
sys.stdout = Logger(logfile_3[0], sys.stdout)

#for i in range(0, len(user_list)):    # 第四级，数据集下的所有user   st_p_1 = [[st_p_1_u], [st_p_1_u]]
for i in range(0, 4):    # 第四级，数据集下的所有user   st_p_1 = [[st_p_1_u], [st_p_1_u]]

    a1 = 0      #记录起始索引位置
    b1 = 0
    st_p_1_u = []     #第一类停留点list
    st_p_2_u = []     #第二类停留点list
    user = user_list[i]
    file_path = path_info + user + "\\Trajectory\\"
    file_list = os.listdir(file_path)
    print("###################    用户： ", user,  "####################################")
    print("用户文件目录： ", file_path)
    print("该用户有轨迹数目： ", len(file_list))
    #print("#st_p_1 第一类停留点总索引开始位置： ", a)
    #print("#st_p_2 第二类停留点总索引开始位置： ", b)
    a1 = a
    b1 = b
    for j in range(0, len(file_list)):       # 第三级 user 下的所有文件 st_p_1_u = [st_p_f, st_p_f]
        a2 = 0
        b2 = 0
        file_name = file_path + file_list[j]
        print("正在处理文件：", file_name)
        
        print("搜索第一类停留点: ")
        tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        tra2 = beijing_6huan(tra1)
        #st_p_1_m = single_1st_stay(tra, 10, 10)       # 第一类停留点 返回三元组list   （平均位置，停留时长）
        [tra, st_p_i] = single_stay_point(tra2, 10, 10)
        st_p_f = []
        for k in range(0, len(st_p_i)):       # 第二级，文件内所有的停留点  st_p_f = [[st_p],[st_p],[st_p]]
            
            st_p = []    
            for m in range(st_p_i[k][0], st_p_i[k][1]+1):        #  第一级，停留点具体轨迹点  st_p = [[st_p_gps],[st_p_gps]]
                
                st_p_gps = [tra.iloc[m][0], tra.iloc[m][1], tra.iloc[m][4]]
                st_p.append(st_p_gps)
            
            st_p_f.append(st_p)
        
        a2 = len(st_p_f)      # 文件中停留点的个数
        a += a2
        print("用户： ",user,"在第：",j+1," 个文件： ",file_list[j],"中有第一类停留点： ",a2,"个")
        
        
        
        
        st_p_1_u.append(st_p_f)
        
        
        
        
        print("搜索第二类停留点: ")
        tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        tra2 = beijing_6huan(tra1)
        
        [tra, st_p_i] = single_stay_point(tra2, 100, 10)
        st_p_f = []
        for k in range(0, len(st_p_i)):       # 第二级，文件内所有的停留点  st_p_f = [[stay_p],[stay_p],[stay_3]]
            
            st_p = []    
            for m in range(st_p_i[k][0], st_p_i[k][1]+1):        #  第一级，停留点具体轨迹点  st_p = [[st_p_gps],[st_p_gps]]
                
                st_p_gps = [tra.iloc[m][0], tra.iloc[m][1], tra.iloc[m][4]]
                st_p.append(st_p_gps)
            
            st_p_f.append(st_p)
        
        b2 = len(st_p_f)      # 文件中停留点的个数
        b += b2
        print("用户： ",user,"在第：",j+1," 个文件： ",file_list[j],"中有第二类停留点： ",b2,"个")
        
        
        
        
        st_p_2_u.append(st_p_f)
        
        
        
        #print("搜索第二类停留点: ")
        #tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        #tra = beijing_6huan(tra1)
        #st_p_2_m = single_2st_stay(tra, 100, 10)      # 第二类停留点 返回五元组list    （区域范围，停留时长）
        #b2 = len(st_p_2_m)
        #b += b2
        #print("用户： ",user," 在第：",j+1," 个文件： ",file_list[j]," 中有第二类停留点： ",b2,"个")
        #st_p_2_user.append(st_p_2_m)

    st_p_1.append(st_p_1_u)
    st_p_2.append(st_p_2_u)
    
    print("用户： ",user, "共有第一类停留点： ",a-a1,"个")
    #print("#st_p_1 总第一类停留点，开始索引位置(行号): ",a1,"结束索引位置： ",a)
    print("用户： ",user, "共有第二类停留点： ",b-b1,"个")
    #print("#st_p_2 总第二类停留点，开始索引位置(行号): ",b1,"结束索引位置： ",b)
    print("###########################################################################")
    
write_sp_txt(st_p_1, filename1)
write_sp_txt(st_p_2, filename2)
print("共提取第一类停留点： ", a, " 个")
print("共提取第二类停留点： ", b, " 个")
print("ALL DONE！！！！！已提取所有停留点，并打印输出，请查收！")

###################    用户：  000 ####################################
用户文件目录：  H:\Geolife Trajectories 1.3\Data\000\Trajectory\
该用户有轨迹数目：  171
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20081023025304.plt
搜索第一类停留点: 
用户：  000 在第： 1  个文件：  20081023025304.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  509 -- 605
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  884 -- 908
用户：  000 在第： 1  个文件：  20081023025304.plt 中有第二类停留点：  2 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20081024020959.plt
搜索第一类停留点: 
用户：  000 在第： 2  个文件：  20081024020959.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  26 -- 79
用户：  000 在第： 2  个文件：  20081024020959.plt 中有第二类停留点：  1 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20081026134407.plt
搜索第一类停留点: 
用户：  000 在第： 3  个文件：  20081026134407.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
用户：  000 在第： 3  个文件：  20081026134407.plt 中有第二类停留点：  0 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajecto

用户：  000 在第： 56  个文件：  20090413050806.plt 中有第二类停留点：  3 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090413144632.plt
搜索第一类停留点: 
用户：  000 在第： 57  个文件：  20090413144632.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
用户：  000 在第： 57  个文件：  20090413144632.plt 中有第二类停留点：  0 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090413213935.plt
搜索第一类停留点: 
用户：  000 在第： 58  个文件：  20090413213935.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  178 -- 205
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  821 -- 824
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  1567 -- 1596
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  1679 -- 1711
出现 stay point！！！ 现有 5 个stay point；   停留位置索引范围为：  1841 -- 1910
用户：  000 在第： 58  个文件：  20090413213935.plt 中有第二类停留点：  5 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090415000406.plt
搜索第一类停留点: 
用户：  000 在第： 59  个文件：  20090415000406.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
用户：  000 在第： 59  个文件：  20090415000406.plt 中有第二类停留点：  0 个
正在处理

用户：  000 在第： 102  个文件：  20090513054336.plt 中有第二类停留点：  4 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090514015425.plt
搜索第一类停留点: 
用户：  000 在第： 103  个文件：  20090514015425.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  57 -- 172
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  222 -- 291
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  483 -- 562
用户：  000 在第： 103  个文件：  20090514015425.plt 中有第二类停留点：  3 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090515014004.plt
搜索第一类停留点: 
用户：  000 在第： 104  个文件：  20090515014004.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  123 -- 215
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  288 -- 355
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  356 -- 396
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  576 -- 892
用户：  000 在第： 104  个文件：  20090515014004.plt 中有第二类停留点：  4 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090516091038.plt
搜索第一类停留点: 
用户：  000 在第

出现 stay point！！！ 现有 9 个stay point；   停留位置索引范围为：  3251 -- 3305
用户：  000 在第： 147  个文件：  20090616054128.plt 中有第二类停留点：  9 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090618005931.plt
搜索第一类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  163 -- 177
用户：  000 在第： 148  个文件：  20090618005931.plt 中有第一类停留点：  1 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  72 -- 94
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  159 -- 184
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  326 -- 369
用户：  000 在第： 148  个文件：  20090618005931.plt 中有第二类停留点：  3 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\000\Trajectory\20090619004036.plt
搜索第一类停留点: 
用户：  000 在第： 149  个文件：  20090619004036.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  226 -- 251
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  252 -- 272
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  404 -- 650
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  651 -- 924
出现 stay point！！！ 现有 5 个stay point；

出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  195 -- 220
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  876 -- 1071
用户：  001 在第： 20  个文件：  20081104054859.plt 中有第二类停留点：  2 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\001\Trajectory\20081104234436.plt
搜索第一类停留点: 
用户：  001 在第： 21  个文件：  20081104234436.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  7 -- 46
用户：  001 在第： 21  个文件：  20081104234436.plt 中有第二类停留点：  1 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\001\Trajectory\20081105110052.plt
搜索第一类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  1264 -- 1784
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  1815 -- 1954
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  3086 -- 3278
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  3669 -- 3813
出现 stay point！！！ 现有 5 个stay point；   停留位置索引范围为：  3834 -- 4006
出现 stay point！！！ 现有 6 个stay point；   停留位置索引范围为：  4287 -- 5196
用户：  001 在第： 22  个文件：  20081105110052.plt 中有第一类停留点：  6 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay 

...

用户：  003 在第： 153  个文件：  20090222052937.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  416 -- 450
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  525 -- 575
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  763 -- 958
用户：  003 在第： 153  个文件：  20090222052937.plt 中有第二类停留点：  3 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090222183204.plt
搜索第一类停留点: 
用户：  003 在第： 154  个文件：  20090222183204.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
用户：  003 在第： 154  个文件：  20090222183204.plt 中有第二类停留点：  0 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090223053653.plt
搜索第一类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  136 -- 149
用户：  003 在第： 155  个文件：  20090223053653.plt 中有第一类停留点：  1 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  134 -- 154
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  190 -- 270
用户：  003 在第： 155  个文件：  20090223053653.plt 中有第二类停留点：  2 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090224003701.plt
搜索第一类停留点: 
用

搜索第一类停留点: 
用户：  003 在第： 175  个文件：  20090308182352.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
用户：  003 在第： 175  个文件：  20090308182352.plt 中有第二类停留点：  0 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090309004221.plt
搜索第一类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  3608 -- 3616
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  4899 -- 4901
用户：  003 在第： 176  个文件：  20090309004221.plt 中有第一类停留点：  2 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  447 -- 504
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  683 -- 705
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  771 -- 803
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  958 -- 1108
出现 stay point！！！ 现有 5 个stay point；   停留位置索引范围为：  1171 -- 1322
出现 stay point！！！ 现有 6 个stay point；   停留位置索引范围为：  1370 -- 1484
出现 stay point！！！ 现有 7 个stay point；   停留位置索引范围为：  1485 -- 1789
出现 stay point！！！ 现有 8 个stay point；   停留位置索引范围为：  2511 -- 2891
出现 stay point！！！ 现有 9 个stay point；   停留位置索引范围为：  2900 -- 3234
出现 stay point！！！ 现有 10 个stay poi

搜索第一类停留点: 
用户：  003 在第： 211  个文件：  20090415074008.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  750 -- 807
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  907 -- 1018
用户：  003 在第： 211  个文件：  20090415074008.plt 中有第二类停留点：  2 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090416002705.plt
搜索第一类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  1262 -- 1281
用户：  003 在第： 212  个文件：  20090416002705.plt 中有第一类停留点：  1 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  40 -- 92
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  271 -- 413
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  579 -- 729
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  1011 -- 1039
出现 stay point！！！ 现有 5 个stay point；   停留位置索引范围为：  1181 -- 1215
出现 stay point！！！ 现有 6 个stay point；   停留位置索引范围为：  1250 -- 1296
出现 stay point！！！ 现有 7 个stay point；   停留位置索引范围为：  1297 -- 1587
出现 stay point！！！ 现有 8 个stay point；   停留位置索引范围为：  1652 -- 1690
用户：  003 在第： 212  个文件：  200904160027

出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  112 -- 170
用户：  003 在第： 256  个文件：  20090516091038.plt 中有第二类停留点：  1 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090517054624.plt
搜索第一类停留点: 
用户：  003 在第： 257  个文件：  20090517054624.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  160 -- 192
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  361 -- 385
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  528 -- 548
用户：  003 在第： 257  个文件：  20090517054624.plt 中有第二类停留点：  3 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090517220626.plt
搜索第一类停留点: 
用户：  003 在第： 258  个文件：  20090517220626.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  141 -- 313
用户：  003 在第： 258  个文件：  20090517220626.plt 中有第二类停留点：  1 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090518064638.plt
搜索第一类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  102 -- 103
用户：  003 在第： 259  个文件：  20090518064638.plt 中有第一类停留点：  1 个
搜索第二类停留点: 
出

出现 stay point！！！ 现有 6 个stay point；   停留位置索引范围为：  1309 -- 1483
用户：  003 在第： 300  个文件：  20090619004036.plt 中有第二类停留点：  6 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090620024527.plt
搜索第一类停留点: 
用户：  003 在第： 301  个文件：  20090620024527.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  389 -- 422
用户：  003 在第： 301  个文件：  20090620024527.plt 中有第二类停留点：  1 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090621101407.plt
搜索第一类停留点: 
用户：  003 在第： 302  个文件：  20090621101407.plt 中有第一类停留点：  0 个
搜索第二类停留点: 
出现 stay point！！！ 现有 1 个stay point；   停留位置索引范围为：  169 -- 187
出现 stay point！！！ 现有 2 个stay point；   停留位置索引范围为：  333 -- 630
出现 stay point！！！ 现有 3 个stay point；   停留位置索引范围为：  631 -- 801
出现 stay point！！！ 现有 4 个stay point；   停留位置索引范围为：  959 -- 1086
出现 stay point！！！ 现有 5 个stay point；   停留位置索引范围为：  1243 -- 1819
用户：  003 在第： 302  个文件：  20090621101407.plt 中有第二类停留点：  5 个
正在处理文件： H:\Geolife Trajectories 1.3\Data\003\Trajectory\20090622095432.plt
搜索第一类停留点: 
出现 st

In [ ]:
####################    总体实验    #########################################
# user = "000"      # 字符串    user user_m    user_f  都是字符串
st_p_1 = []     #第一类停留点list
st_p_2 = []     #第二类停留点list
a = 0
b = 0
path_info = "H:\Geolife Trajectories 1.3\Data\\"
#user_m = str(int(user)+1) 
#user_f = user_m.zfill(3)
user_list = os.listdir(path_info)

filename1 = ".\\Stay_point\\" + "st_p_1-" +str(time.strftime("%Y%m%d-%H%M%S")) + '.txt'
filename2 = ".\\Stay_point\\" + "st_p_2-" +str(time.strftime("%Y%m%d-%H%M%S")) + '.txt'
logfile_3 =[".\\Stay_poitn\\" + str(time.strftime("%Y%m%d-%H%M%S")) + '.log']
sys.stdout = Logger(logfile_3[0], sys.stdout)

for i in range(0, len(user_list)):

    a1 = 0      #记录起始索引位置
    b1 = 0
    st_p_1_user = []     #第一类停留点list
    st_p_2_user = []     #第二类停留点list
    user = user_list[i]
    file_path = path_info + user + "\\Trajectory\\"
    file_list = os.listdir(file_path)
    print("###################    用户： ", user,  "####################################")
    print("用户文件目录： ", file_path)
    print("该用户有轨迹数目： ", len(file_list))
    #print("#st_p_1 第一类停留点总索引开始位置： ", a)
    #print("#st_p_2 第二类停留点总索引开始位置： ", b)
    a1 = a
    b1 = b
    for j in range(0, len(file_list)):       #对file_list（user） 下的每一个文件
        a2 = 0
        b2 = 0
        file_name = file_path + file_list[j]
        print("正在处理文件：", file_name)
        
        print("搜索第一类停留点: ")
        tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        tra2 = beijing_6huan(tra1)
        st_p_1_m = single_1st_stay(tra, 10, 10)       # 第一类停留点 返回三元组list   （平均位置，停留时长）
        a2 = len(st_p_1_m)
        a += a2
        print("用户： ",user,"在第：",j+1," 个文件： ",file_list[j],"中有第一类停留点： ",a2,"个")
        st_p_1_user.append(st_p_1_m)
        print("搜索第二类停留点: ")
        tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        tra = beijing_6huan(tra1)
        st_p_2_m = single_2st_stay(tra, 100, 10)      # 第二类停留点 返回五元组list    （区域范围，停留时长）
        b2 = len(st_p_2_m)
        b += b2
        print("用户： ",user," 在第：",j+1," 个文件： ",file_list[j]," 中有第二类停留点： ",b2,"个")
        st_p_2_user.append(st_p_2_m)

    st_p_1.append(st_p_1_user)
    st_p_2.append(st_p_2_user)
    
    print("用户： ",user, "共有第一类停留点： ",a-a1,"个")
    #print("#st_p_1 总第一类停留点，开始索引位置(行号): ",a1,"结束索引位置： ",a)
    print("用户： ",user, "共有第二类停留点： ",b-b1,"个")
    #print("#st_p_2 总第二类停留点，开始索引位置(行号): ",b1,"结束索引位置： ",b)
    print("###########################################################################")
    
write_sp_txt(st_p_1, filename1)
write_sp_txt(st_p_2, filename2)
print("共提取第一类停留点： ", a, " 个")
print("共提取第二类停留点： ", b, " 个")
print("ALL DONE！！！！！已提取所有停留点，并打印输出，请查收！")
       

In [ ]:
####################    提取10个用户的stay_point    #########################################3
#user = "000"      # 字符串    user user_m    user_f  都是字符串
st_p_1 = []     #第一类停留点list
st_p_2 = []     #第二类停留点list
a = 0
b = 0
path_info = "H:\Geolife Trajectories 1.3\Data\\"
#user_m = str(int(user)+1) 
#user_f = user_m.zfill(3)
user_list = os.listdir(path_info)

filename1 = ".\\Stay_point\\" + "st_p_1_10-" +str(time.strftime("%Y%m%d-%H%M%S")) + '.txt'
filename2 = ".\\Stay_point\\" + "st_p_2_10-" +str(time.strftime("%Y%m%d-%H%M%S")) + '.txt'
#for i in range(0, len(user_list)):
for i in range(0, 10):
    a1 = 0
    b1 = 0
    st_p_1_user = []     #第一类停留点list
    st_p_2_user = []     #第二类停留点list
    user = user_list[i]
    file_path = path_info + user + "\\Trajectory\\"
    file_list = os.listdir(file_path)
    print("###################    用户： ", user,  "####################################")
    print("用户文件目录： ", file_path)
    print("该用户有轨迹数目： ", len(file_list))
    #print("#st_p_1 第一类停留点总索引开始位置： ", a)
    #print("#st_p_2 第二类停留点总索引开始位置： ", b)
    a1 = a
    b1 = b
    for j in range(0, len(file_list)):
        a2 = 0
        b2 = 0
        file_name = file_path + file_list[j]
        print("正在处理文件：", file_name)
        
        print("搜索第一类停留点: ")
        tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        tra = beijing_6huan(tra1)
        st_p_1_m = single_1st_stay(tra, 10, 10)       # 第一类停留点 返回三元组list   （平均位置，停留时长）
        a2 = len(st_p_1_m)
        a += a2
        print("用户： ",user,"在第：",j+1," 个文件： ",file_list[j],"中有第一类停留点： ",a2,"个")
        st_p_1_user.append(st_p_1_m)
        print("搜索第二类停留点: ")
        tra1 = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
        tra = beijing_6huan(tra1)
        st_p_2_m = single_2st_stay(tra, 100, 10)      # 第二类停留点 返回五元组list    （区域范围，停留时长）
        b2 = len(st_p_2_m)
        b += b2
        print("用户： ",user," 在第：",j+1," 个文件： ",file_list[j]," 中有第二类停留点： ",b2,"个")
        st_p_2_user.append(st_p_2_m)

    st_p_1.append(st_p_1_user)
    st_p_2.append(st_p_2_user)
    
    print("用户： ",user, "共有第一类停留点： ",a-a1,"个")
    #print("#st_p_1 总第一类停留点，开始索引位置(行号): ",a1,"结束索引位置： ",a)
    print("用户： ",user, "共有第二类停留点： ",b-b1,"个")
    #print("#st_p_2 总第二类停留点，开始索引位置(行号): ",b1,"结束索引位置： ",b)
    print("###########################################################################")
    
print("共提取第一类停留点： ", a, " 个")
print("共提取第二类停留点： ", b, " 个")
write_sp_txt(st_p_1, filename1)
write_sp_txt(st_p_2, filename2)
print("ALL DONE！！！！！已提取所有停留点，并打印输出，请查收！")
       

In [27]:
###################      单用户测试            ##################
file_path = "H:\Geolife Trajectories 1.3\Data\\001\\Trajectory\\"
#user_m = str(int(user)+1) 
#user_f = user_m.zfill(3)
#user_list = os.listdir(path_info)

st_p_1 = []     #第一类停留点list
st_p_2 = []     #第二类停留点list

file_list = os.listdir(file_path)
#print("打印用户", user)
print("用户文件目录", file_path)
print("该用户有轨迹数目：", len(file_list))
for j in range(0, len(file_list)):
    #st_p_1_m = []
    #st_p_2_m = []
    file_name = file_path + file_list[j]
    print("正在处理文件：", file_name)

    print("搜索第一类停留点")
    tra = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
    tra = beijing_6huan(tra)
    st_p_1_m = single_1st_stay(tra, 10, 10)       # 第一类停留点 返回三元组list   （平均位置，停留时长）
    st_p_1.append(st_p_1_m)
    print("搜索第二类停留点")
    tra = pd.read_csv(file_name,skiprows=[0,1,2,3,4,5], header = None)
    tra = beijing_6huan(tra)
    st_p_2_m = single_2st_stay(tra, 100, 10)      # 第二类停留点 返回五元组list    （区域范围，停留时长）
    st_p_2.append(st_p_2_m)
    
    #tra.drop(tra.index,inplace=True)        #   清空轨迹集
    
print("该用户有第一类停留点",len(st_p_1),"个","\n","第二类停留点",len(st_p_2),"个")
print("打印第一类停留点：",st_p_1)
print("打印第二类停留点：",st_p_2)
        
   # print(path_user)
#len(user_list)

In [15]:

#####################   单文件测试     ####################################

path = 'H:\Geolife Trajectories 1.3\Data\\000\\Trajectory\\'
#file_list = os.listdir(path)
# file_list 
#i = 0
#filename = path + file_list[3]
filename = path + "20090331003904.plt"
#print(filename)
tra1 = pd.read_csv(filename,skiprows=[0,1,2,3,4,5], header = None)
tra = beijing_6huan(tra1)

st_p_2 = single_2st_stay(tra, 100, 10)
print(st_p_2)

#tra[1].dtype
#len(tra)

In [ ]:
path = 'H:\Geolife Trajectories 1.3\Data\\000\\Trajectory\\'
#file_list = os.listdir(path)
# file_list 
#i = 0
#filename = path + file_list[3]
filename = path + "20090331003904.plt"
tra = pd.read_csv(filename,skiprows=[0,1,2,3,4,5], header = None)
len(tra)

In [34]:
st_p = single_2st_stay(tra, 100, 10)
st_p

Moving points, 下阶段索引起始位置为：  2
Moving points, 下阶段索引起始位置为：  12
Moving points, 下阶段索引起始位置为：  19
出现 stay point,现共有： 1 个stay point；   位置索引范围为：  19 -- 72 
 0        40.009
1       116.322
2             0
3           115
4       39745.1
5    2008-10-24
6      02:22:04
Name: 72, dtype: object
本文件执行到索引：  243 ；  len(tra) =  244 ；  共有异常点： 0 个


[[40.00909, 40.00861, 116.322134, 116.32119099999998, 10.500000055180863]]

In [31]:
f = mean_point(st_p_l)
f

[40.00877419999999, 116.322371]

In [ ]:
#stay_point = np.array([[ 0,0 ]])       # 创建stay_point   array, # 第一行 [0, 0] 作为标题行。
#c = np.array([[i,j-1]]）               # 把每个stay point的起止位置索引作为一行，并入;
#stay_point = np.r_[stay_point,c]        #  实现 np.r_[] ：把两个array合并

In [37]:
l = [[40.008692,116.322194],[40.008829,116.322251]] 
L = pd.DataFrame(l) 
L

,0,1
0,40.008692,116.322194
1,40.008829,116.322251


In [38]:
Dis(L)

15.988549952594987

In [33]:
a = [39745.1037152778,39745.1122222222]
A = pd.DataFrame(a) 
A

,0
0,39745.103715
1,39745.112222


In [34]:
Int(A)

0    12.25
dtype: float64